In [172]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import pandas as pd


In [177]:
# Pretend to be a web browser and make a get request of a webpage
wine_request = get('https://www.wine.com/list/wine/7155')



In [178]:
wine_request.status_code

200

In [208]:
global product

def read_wine_page(wine_request):
    # The .text returns the text from the request
    wine_html = wine_request.text
    # Then make the soup, specifying the "lxml" parser
    house_soup = BeautifulSoup(wine_html,'lxml')
    mydivs = house_soup.findAll("div", {"class": "prodItem_wrap"})

    wineNames = []
    wineImgs = []
    winePrices = []
    wineUrls = []
    raw_htmls = []
    
    for product in mydivs:
        wineNames.append(product.find('span', {'class' : 'prodItemInfo_name'}).text)
        wineImgs.append(str(product.find('img')).split('src=')[1].split('/>')[0].replace('"', ''))
        winePrices.append(float(str(product.find('div', {'class' : 'prodItemStock_addCart'}))\
                                    .split('data-unit=')[1]\
                                    .split('>')[0]\
                                    .replace('"', '')\
                                    .replace('$','')\
                                    .replace(',','')))
        wineUrls.append(product.find('a', {'class' : 'prodItemImage_link'})['href'])
        raw_htmls.append(str(product))
    
    
    winePage_df = pd.DataFrame(data=[wineNames,
                                     wineImgs,
                                     winePrices,
                                     wineUrls,
                                     raw_htmls]).transpose()
    winePage_df.columns = ['name','image_path','price','url','raw_html']
    
    return winePage_df


In [195]:
product.find('div', {'class' : 'prodItemStock_addCart'})['data-unit']

'$44.99'

In [200]:
winePage_df.head()


,Name,Image_Link,Price
0,Hall Napa Valley Cabernet Sauvignon 2013,/product/images/fl_progressive/158457.jpg,54.99
1,Rombauer Chardonnay 2017,/product/images/fl_progressive/wpxkrgx7onirby1...,36.99
2,Chateau Cos d'Estournel (Futures Pre-Sale) 2016,/product/images/fl_progressive/202293.jpg,219.97
3,Silver Oak Napa Valley Cabernet Sauvignon 2014,/product/images/fl_progressive/fog0sb2avopbbq0...,119.99
4,Silver Oak Alexander Valley Cabernet Sauvignon...,/product/images/fl_progressive/gpgmugpbf3l6is2...,74.99


In [207]:
type(wine_master_df)

tuple

In [218]:
wine_page_ix = 1
while True:
    if wine_page_ix == 1:
        get_response = get('https://www.wine.com/list/wine/7155')
        wine_master_df = read_wine_page(get_response)
        wine_page_ix += 1
        
    # Pull request and check if OK response
    get_response = get('https://www.wine.com/list/wine/7155/{}'.format(wine_page_ix))
    temp_df = read_wine_page(get_response)
    if get_response.status_code != 200 or temp_df.shape[0] < 20:
        print("Finished pulling data, found {} status code".format(get_response.status_code))
        break
    
    # Append results
    wine_master_df = pd.concat([wine_master_df,temp_df], axis=0)
    print("{}: df shape: {}".format(wine_page_ix, wine_master_df.shape))
    wine_page_ix += 1

2: df shape: (50, 5)
3: df shape: (75, 5)
4: df shape: (100, 5)
5: df shape: (125, 5)
6: df shape: (150, 5)
7: df shape: (175, 5)
8: df shape: (200, 5)
9: df shape: (225, 5)
10: df shape: (250, 5)
11: df shape: (275, 5)
12: df shape: (300, 5)
13: df shape: (325, 5)
14: df shape: (350, 5)
15: df shape: (375, 5)
16: df shape: (400, 5)
17: df shape: (425, 5)
18: df shape: (450, 5)
19: df shape: (475, 5)
20: df shape: (500, 5)
21: df shape: (525, 5)
22: df shape: (550, 5)
23: df shape: (575, 5)
24: df shape: (600, 5)
25: df shape: (625, 5)
26: df shape: (650, 5)
27: df shape: (675, 5)
28: df shape: (700, 5)
29: df shape: (725, 5)
30: df shape: (750, 5)
31: df shape: (775, 5)
32: df shape: (800, 5)
33: df shape: (825, 5)
34: df shape: (850, 5)
35: df shape: (875, 5)
36: df shape: (900, 5)
37: df shape: (925, 5)
38: df shape: (950, 5)
39: df shape: (975, 5)
40: df shape: (1000, 5)
41: df shape: (1025, 5)
42: df shape: (1050, 5)
43: df shape: (1075, 5)
44: df shape: (1100, 5)
45: df shape: (1

In [219]:
wine_master_df.to_pickle('data/scrape_v2.pickle')

In [220]:
wine_master_df.drop_duplicates().shape

(15546, 5)